In [1]:
import pandas as pd
import os
import uuid
from email import generator
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import re

In [20]:
# where to write the output file
directory = "../EMAILS"

In [36]:
canceled = pd.read_csv("../canceled.csv", delimiter =";").dropna()
canceled.head()

,DataReg,Rag,AAMS,Importo,authmail,SaldoCassa,SaldoProg,SaldoDov
0,17-May,val7,val7,"3.000,00 €",dir6@mycompany,19062.0,3246.0,0.0
1,17-May,val5,val5,"2.000,00 €",ref5@external,5640.0,-33068.0,-20164.0
2,17-May,val32,val32,"2.000,00 €",ref6@external,7923.0,-18083.0,-12378.0
3,17-May,val29,val29,"21.000,00 €",ref5@external,44869.0,-49852.0,-27706.0
4,17-May,val25,val25,"3.000,00 €",ref14@external,37120.0,-28485.0,-6525.0


In [29]:
template_dir="""
Buonasera,

ti informiamo che in data odierna è stato cancellato un versamento di {} inserito sul conto del PV in data {}.
Di seguito situazione aggiornata

Ragione Sociale     :    {}
Saldo Attuale       :    {}
Saldo Dovuto        :    {}
Saldo Cassa         :    {}


 
Saluti
MyCompany 
 
"""


In [30]:
template_ref="""
Gentile Referente,

La informiamo che in data odierna è stato cancellato un versamento di {} inserito sul conto del PV in data {}.
Di seguito situazione aggiornata

Ragione Sociale     :    {}
Saldo Attuale       :    {}
Saldo Dovuto        :    {}
Saldo Cassa         :    {}


 
Saluti
MyCompany  
 
 """

In [34]:
def compose_emails(data):
    
    # splits datasets based on authmail address, it can be set to specific email domain for internal addresses such as @RandomCompany
    
    directs = [canceled.loc[n] for n in range(len(canceled)) if canceled.loc[n].authmail.endswith('@mycompany')==True]
    ref = [canceled.loc[n] for n in range(len(canceled)) if canceled.loc[n].authmail.endswith('@mycompany')==False]
    dir_pv = pd.DataFrame(directs).reset_index(drop=True)
    ref_pv = pd.DataFrame(ref).reset_index(drop=True)
    
    def create_message():
        msg = MIMEMultipart()
        msg["To"] = address_to
        msg["From"] = address_from
        msg["Subject"] = subject
        msg.attach(MIMEText(body, "plain"))

        return msg

    def write_eml_file_dir(msg):
        os.chdir(directory)
        filename = 'DIR' + str(uuid.uuid4()) + ".eml"

        with open(filename, 'w') as file:
            emlGenerator = generator.Generator(file)
            emlGenerator.flatten(msg)
    def write_eml_file_ref(msg):
        os.chdir(directory)
        filename = 'REF' + str(uuid.uuid4()) + ".eml"

        with open(filename, 'w') as file:
            emlGenerator = generator.Generator(file)
            emlGenerator.flatten(msg)
    
    print('Payments canceled for DIR PV:  ' + str(len(dir_pv)))

    print('Payments canceled for REF PV:  ' + str(len(ref_pv)))

    #. Direct PVs email composition
    n=0
    for n in range(len(dir_pv)):
        item = dir_pv.loc[n]
        address_from = "m.amanti@mycompany.com"
        address_to = item.authmail
        amount = item.Importo
        date = item.DataReg
        pv_name = item.Rag
        balance_current = item.SaldoProg
        balance_due = item.SaldoDov
        balance_cassa = item.SaldoCassa
        subject = "Bonifico cancellato - Mancato incasso {} - {}".format(amount,pv_name)
        body = template_dir.format(amount,date,pv_name,balance_current,balance_due,balance_cassa)
        n+=1
        
   
                
        msg = create_message()
        write_eml_file_dir(msg)
      
    
    #. Ref PVs email composition
    n=0
    for n in range(len(ref_pv)):
        item = ref_pv.loc[n]
        address_from = "generic@mycompany.com"
        address_to = item.authmail
        amount = item.Importo
        date = item.DataReg
        pv_name = item.Rag
        balance_current = item.SaldoProg
        balance_due = item.SaldoDov
        balance_cassa = item.SaldoCassa
        subject = "Bonifico cancellato - Mancato incasso {} - {}".format(amount,pv_name)
        body = template_ref.format(amount,date,pv_name,balance_current,balance_due,balance_cassa)
        n+=1
        
       
                
        msg = create_message()
        write_eml_file_ref(msg)

In [35]:
compose_emails(canceled)

Payments canceled for DIR PV:  10
Payments canceled for REF PV:  22
